In [38]:
!pip install -U dspy-ai


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [77]:
import os
import json
import dspy
import re

In [78]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [79]:
from pydantic import BaseModel, Field

In [80]:
class GameOf24(dspy.Signature):
    """
    Finalize the mathematical expression for the Game of 24.
    Given the original numbers and the successful reasoning steps, 
    format the final answer as a single mathematical expression.
    """
    problem = dspy.InputField(desc="The initial four numbers")
    reasoning_steps = dspy.InputField(desc="The sequence of valid moves found")
    expression = dspy.OutputField(desc="The final formula using parentheses, e.g., (4 * 3) * (1 + 1)")

In [81]:

# Enable DSPy internal logging
dspy.enable_logging()

class MathStep(BaseModel):
    num1: int = Field(description="First number")
    operator: str = Field(description="+, -, *, or /")
    num2: int = Field(description="Second number")
    result: int = Field(description="The STRICT mathematical result of (num1 operator num2)")
    remaining: list[int] = Field(description="New list of numbers")
    
  
class ProposeThoughts(dspy.Signature):
    """Propose up to 10 distinct mathematical moves.
    STRICT RULES:
    1. You MUST ONLY use numbers from the 'available_numbers' list.
    2. Do NOT use numbers like 6, 8, 12, or 24 if they are not in the list.
    3. 'result' MUST be the actual math (e.g., if you pick 1 and 2, result is 3).
    4. Format the 'proposals' field as a clean JSON list of MathStep objects.
    """
    available_numbers: list[int] = dspy.InputField()
    target: int = dspy.InputField()
    proposals: list[MathStep] = dspy.OutputField()

    
# 1. Use Pydantic for structured scoring
class Evaluation(BaseModel):
    score: float = Field(description="A numeric score between 1 and 10")
    reasoning: str = Field(description="Brief explanation of the score")

class EvaluateThought(dspy.Signature):
    """
    Evaluate if a mathematical move brings us closer to the target.
    Score 10: The move hits the target exactly.
    Score 7-9: The move creates a number that is a factor or multiple of the target.
    Score 1: The move creates a large prime or number that makes the target impossible.
    OUTPUT ONLY the raw assessment data. 
    DO NOT include 'Score:' labels or conversational text.
    """
    target: str = dspy.InputField(desc="The number we are trying to reach")
    thought: str = dspy.InputField(desc="The specific math move being scored")
   
    assessment: Evaluation = dspy.OutputField(desc="Score and reasoning") # DSPy handles the parsing automatically


def validate_move(parsed, available):
    """The Ground Truth: Python checks if the LLM's math is actually valid."""
    nums = available.copy()
    try:
        nums.remove(parsed["a"])
        nums.remove(parsed["b"])
    except ValueError:
        return False, [] # Numbers not available
        
    ops = {
        '+': lambda x, y: x + y, 
        '-': lambda x, y: x - y, 
        '*': lambda x, y: x * y, 
        '/': lambda x, y: x / y if y != 0 else None
    }
    
    calc_res = ops.get(parsed['op'], lambda x, y: None)(parsed['a'], parsed['b'])
    
    if calc_res is None or abs(calc_res - parsed['result']) > 0.001:
        return False, [] # Math is wrong
        
    nums.append(int(parsed["result"]))
    return True, sorted(nums)  
 

# 2. Build the ToT Module
class TreeOfThoughts(dspy.Module):
    def __init__(self, depth=2):
        super().__init__()
        self.proposer = dspy.ChainOfThought(ProposeThoughts)
         # Use standard Predict; it will honor the Type Hint in the Signature
        self.evaluator = dspy.ChainOfThought(EvaluateThought) 
        # ADD THIS: The final solver
        self.solver = dspy.Predict(GameOf24) 
        self.depth = depth

    def forward(self, problem,target):
        target_val = float(target)
        # Start with the initial problem state
        # SEED THE STATE: Make it impossible for the model to miss the numbers
        
        # Extract numbers from the problem string
        initial_nums = [int(x.strip()) for x in problem.split(",") if x.strip().isdigit()]
        
        candidates = [{"steps": f"Goal: {target}", "available": initial_nums, "score": 0.0}]

        for d in range(self.depth):
            print(f"depth is:+{d}")
            new_candidates = []
            for candidate in candidates:
                 # 1. Get structured proposals
                res = self.proposer(
                    available_numbers=candidate["available"],
                    target=target,
                    current_steps=candidate["steps"]
                )
                if not res.proposals: continue
                    
                for move in res.proposals:
                    # Check if 'move' is a valid MathStep with all required attributes
                    if not all(hasattr(move, attr) for attr in ["num1", "operator", "num2", "result"]):
                        print(f"⚠️ Skipping incomplete move: {move}")
                        continue
                    # No parsing needed! We use the Pydantic fields directly.
                    parsed = {
                        "a": move.num1,
                        "op": move.operator,
                        "b": move.num2,
                        "result": float(move.result)
                    }

                    # 2. Python Validation (The Truth)
                    #  uses validate_move function
                    # Get the TRUTH from Python, not the LLM's hallucinated list
                    is_valid, actual_remaining = validate_move(parsed, candidate["available"])

                    if is_valid:
                        move_str = f"{parsed['a']} {parsed['op']} {parsed['b']} = {parsed['result']}"
                        current_path = candidate["steps"] + f" -> {move_str}"
                    
                        print(f"✅ Logic Pass: {move_str} (Remaining: {actual_remaining})")
                        print(f"Evaluating valid move: {move_str} (Remaining: {actual_remaining})")
       
                    
                        # If the result is 24, we don't need to score it or keep searching.
                        # We found a winner!
                        # 3. Success Check
                        if abs(parsed["result"] - target_val) < 0.001 and len(actual_remaining) == 1:
                            print(f"🎯 TARGET FOUND: {move_str}")
                            final = self.solver(problem=problem, reasoning_steps=current_path)
                            return dspy.Prediction(answer=final.expression, path=current_path, score=10.0)
                        
                        # 4. Evaluate and keep searching
                        eval_res = self.evaluator(target=str(target), thought=move_str)
                        new_candidates.append({
                            "steps": current_path,
                            "available": actual_remaining,
                            "score": float(eval_res.assessment.score)
                        })
                    else:
                        print(f"❌ Hallucination Detected: {parsed['a']} {parsed['op']} {parsed['b']} "
                              f"(Numbers {parsed['a']} or {parsed['b']} not in {candidate['available']})")

            # Selection step: Keep the top 4 best-scored paths (Beam Search)
            candidates = sorted(new_candidates, key=lambda x: x["score"], reverse=True)[:4]
            
        # After all depth levels are explored
        if not candidates:
            # Use 'path' and 'answer' so the print statement doesn't crash
            return dspy.Prediction(
                path="No valid math found", 
                answer="N/A", 
                score=0.0
            )
            

        # Use the Solver to wrap it all up
        best = candidates[0]
        if "24" in str(best["steps"]):
            # Use the solver to clean up the formatting of the successful path
            result = self.solver(problem=problem, reasoning_steps=best["steps"])
            return dspy.Prediction(answer=result.expression, path=best["steps"], score=10.0)
            
        result = self.solver(problem=problem, reasoning_steps=best["steps"])
        return dspy.Prediction(answer=result.expression, path=best["steps"], score=best["score"])
        

In [85]:
local_lm = dspy.LM('ollama_chat/'+MODEL, api_base='http://localhost:11434', api_key='')
local_lm.history = [] 
# Set it globally
#dspy.settings.configure(lm=local_lm)
dspy.settings.configure(lm=local_lm, cache=False) 


solver = TreeOfThoughts(depth=3)
problem_input = "1, 5, 6, 3"
result = solver(problem=problem_input,target=24)

print("Best Path:", result.path)
print("Equation:", result.answer)
print("Score:", result.score)


depth is:+0
❌ Hallucination Detected: 1 + 23 (Numbers 1 or 23 not in [1, 5, 6, 3])
❌ Hallucination Detected: 5 + 19 (Numbers 5 or 19 not in [1, 5, 6, 3])
❌ Hallucination Detected: 6 + 18 (Numbers 6 or 18 not in [1, 5, 6, 3])
❌ Hallucination Detected: 3 + 21 (Numbers 3 or 21 not in [1, 5, 6, 3])
❌ Hallucination Detected: 2 + 22 (Numbers 2 or 22 not in [1, 5, 6, 3])
❌ Hallucination Detected: 12 * 12 (Numbers 12 or 12 not in [1, 5, 6, 3])
✅ Logic Pass: 6 * 3 = 18.0 (Remaining: [1, 5, 18])
Evaluating valid move: 6 * 3 = 18.0 (Remaining: [1, 5, 18])
❌ Hallucination Detected: 4 + 10 (Numbers 4 or 10 not in [1, 5, 6, 3])
❌ Hallucination Detected: 3 - 20 (Numbers 3 or 20 not in [1, 5, 6, 3])
❌ Hallucination Detected: 1 - 23 (Numbers 1 or 23 not in [1, 5, 6, 3])
depth is:+1
❌ Hallucination Detected: 1 * 1 (Numbers 1 or 1 not in [1, 5, 18])
✅ Logic Pass: 1 * 5 = 5.0 (Remaining: [5, 18])
Evaluating valid move: 1 * 5 = 5.0 (Remaining: [5, 18])
❌ Hallucination Detected: 1 * 6 (Numbers 1 or 6 not in

In [86]:
num_calls = len(local_lm.history)
print(f"Total prompt calls made: {num_calls}")

Total prompt calls made: 13


In [87]:
dspy.inspect_history(n=15) # Set n high to catch everything





[2026-02-12T19:09:58.734492]

System message:

Your input fields are:
1. `available_numbers` (list[int]): 
2. `target` (int):
Your output fields are:
1. `reasoning` (str): 
2. `proposals` (list[MathStep]):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## available_numbers ## ]]
{available_numbers}

[[ ## target ## ]]
{target}

[[ ## reasoning ## ]]
{reasoning}

[[ ## proposals ## ]]
{proposals}        # note: the value you produce must adhere to the JSON schema: {"type": "array", "$defs": {"MathStep": {"type": "object", "properties": {"num1": {"type": "integer", "description": "First number", "title": "Num1"}, "num2": {"type": "integer", "description": "Second number", "title": "Num2"}, "operator": {"type": "string", "description": "+, -, *, or /", "title": "Operator"}, "remaining": {"type": "array", "description": "New list of numbers", "items": {"type": "integer"}, "title": "Remaining"}, "result": {"type": "integer", "descrip